In [ ]:
import sys
extra_pathes = (
    './src', 
    '../egrid_pkg/src', 
    '../graphparser/src')
for path in extra_pathes:
    if path not in sys.path:
        sys.path.append(path)
import dssex

### Decorating Elements

| in graphic | in footer | element | role | attributes |
|------------|-----------|---------|------|------------|
|at terminal<ul><li>P=0.72</li><li>P.cost=14</li></ul>|#.PValue(id_of_batch=a P=72 direction=-1 cost=27)|PValue|Value of (measured) active power.|<ul><li>id_of_batch</li><li>P</li><li>direction</li><li>cost</li></ul>|
|at terminal<ul><li>Q=0.27</li><li>Q.cost=17.8</li></ul>|#.QValue(id_of_batch=b Q=0.27 cost=27)|QValue|Value of (measured) reactive power.|<ul><li>id_of_batch</li><li>Q</li><li>direction</li><li>cost</li></ul>|
|at terminal<ul><li>I=12.3</li></ul>|#.IValue(id_of_batch=c I=12.3)|IValue|Value of (measured) current.|<ul><li>id_of_batch</li><li>I</li></ul>|
|at node<ul><li>V=1.03</li></ul>|#.Vvalue(id_of_node=c V=1.03)|Vvalue|Value of (measured) voltage.|<ul><li>id_of_node</li><li>V</li></ul>|
||#.Defk(id=kp type=const)|Factor|dinjection efine scaling factor(s)|<ul><li>id</li><li>type</li><li>id_of_source</li><li>value</li><li>min</li><li>max</li><li>is_discrete</li><li>m</li><li>n</li><li>step</li><li>cost</li></ul>|
||#.Klink(id_of_injection=load part=p id_of_factor=kp)|Injectionlink|logical connection <p/>etween injection and a factor|<ul><li>id_of_injection</li><li>part</li><li>id_of_factor</li><li>step</li></ul>|
||#.Deft(id=tap value=0 m=-.00625 n=1 min=-16 max=16)|Factor|define tap factors|<ul><li>id</li><li>type</li><li>id_of_source</li><li>value</li><li>min</li><li>max</li><li>is_discrete</li><li>m</li><li>n</li><li>step</li><li>cost</li></ul>|
|at<ul><li>terminal </li></ul>Tlink=tap|#.Tlink(id_of_node=n0 id_of_branch=tr id_of_factor=tap)|Terminallink|logical connection between a<p/>terminal of a branch and a factor|<ul><li>id_of_node</li><li>id_of_branch</li><li>id_of_factor</li><li>step</li></ul>|
|at node<ul><li>Vlimit.min=.95</li><li>Vlimit.max=1.05</li></ul>|#.Vlimit(min=.95 max=1.05)|Vlimit|limit of voltage|<ul><li>id_of_node</li><li>min</li><li>max</li><li>step</li></ul>|
||#.Defvl(id_of_node=n0 min=.95 step(1 2))|Vlimit|limit of voltage|<ul><li>id_of_node</li><li>min</li><li>max</li><li>step</li></ul>|
||#.Defoterm|Term|term of objective function|<ul><li>fn</li><li>args</li><li>weight</li><li>step</li></ul>|

to be continued




In [ ]:
schema = """
                   (                                 ~~~
               +-->(                                 ~~~
slack +--------+   (-Branch-------------+ n +-------||||| heating_
        Tlink=tap     y_lo=1k-1kj         |                P10=200
                      y_tr=1µ+1µj         |
                                          |        \<-+->/
                                          |           |
                        _cap ||---------+ n +-------((~)) motor_
                          Q10=-10                          P10=160
                          Exp_v_q=2                        Q10=10

#.Deft(id=tap type=var min=-16 max=16 value=0 is_discrete=True cost=.03)
#.Defk(id=kcap type=var min=0 max=5 value=0 is_discrete=True cost=.05)
#.Klink(id_of_injection=cap id_of_factor=kcap part=q)
#.Vlimit(min=.95)
"""
step_params=[
    # first step: minimize voltage violations
    dict(objectives='U', constraints='U'),
    # second step: minimize cost
    dict(objectives='CL', constraints='U', floss=2)]
result = dssex.estimate(schema, step_params=step_params)

In [ ]:
result['messages']

In [ ]:
result['nodes']

In [ ]:
result['branches']

In [ ]:
result['injections']